# Cálculo do VaR para uma carteira de ações

In [1]:
!pip install yfinance
!pip install yahoofinancials
import yfinance as yf
import pandas as pd
import numpy as np
import math as mt

Após a instalação do API yfinance, definimos uma variável para o Ticker de cada ação a fim de facilitar a utilização desta informação nos trechos seguintes do programa.

Em seguida, fora extraido os dados históricos dos papéis escolhidos para um período desejado e podemos observar que nesta parte já utilizamos a variável definida para o Ticker do papel.

In [2]:
CCR = yf.Ticker('CCRO3.SA')
BB = yf.Ticker('BBAS3.SA')
JBS = yf.Ticker('JBSS3.SA')
TIM = yf.Ticker('TIMS3.SA')

CCR_data = CCR.history( start ='2021-06-30', end = '2022-07-01')
BB_data = BB.history(  start ='2021-06-30', end = '2022-07-01')
JBS_data = JBS.history( start ='2021-06-30', end = '2022-07-01')
TIM_data = TIM.history( start ='2021-06-30', end = '2022-07-01')


Inicialmente, calculamos o volume na carteira dos respectivos pepéis e a partir da cotações diárias de fechamento de cada um deles e das suas quantidades em carteira, que neste estudo será de 1000 cada ativo.

Em seguida, obtemos as posições diárias da carteira e os devidos retornos, o quais usaremos para o cálculo do VaR.

In [3]:
Qt = 1000


#Cálculo das posições dos ativos
CCR_data['PosicaoC'] = CCR_data['Close']*Qt
BB_data['PosicaoB'] = BB_data['Close']*Qt
JBS_data['PosicaoJ'] = JBS_data['Close']*Qt
TIM_data['PosicaoT'] = TIM_data['Close']*Qt


#Criação de DataFrame das posições diárias da carteira, que é a soma da posição dos ativos
Posicao = pd.DataFrame(CCR_data['PosicaoC']+BB_data['PosicaoB']+JBS_data['PosicaoJ']+TIM_data['PosicaoT'], 
                       columns = ['Posicao'])


#Retorno calculado pela diferença entre a posição de um dia e a do dia anterior

Posicao['Retorno'] = (Posicao['Posicao'].shift(1) - Posicao['Posicao']).fillna(0)
Posicao


,Posicao,Retorno
Date,,
2021-06-30,79556.023598,0.000000
2021-07-01,79199.604034,356.419563
2021-07-02,79872.173309,-672.569275
2021-07-05,79561.988831,310.184479
2021-07-06,78117.422104,1444.566727
...,...,...
2022-06-24,90206.725121,-351.645470
2022-06-27,91028.490067,-821.764946
2022-06-28,91247.765541,-219.275475


Como estou considerando um nível de confiança de 95%, podemos encontrar o VaR através do percentil 5%, que é o valor que poderá se superado em 5% dos caso.

In [4]:
var = round(np.percentile(Posicao['Retorno'],5)*(-1),2)
var

1931.94

Calculado o VaR, precisamos agora fazer o Back Testing do modelo utilizado, com o objetivo de validá-lo. Ele é realizado 
a partir do levantamento da quantidade de dias que excederam a perda estimada no VaR, e com isso, espera-se que estejam em
uma porcentagem igual ou aproximada ao nível de significância, que neste estudo é de 5%.

Para comparação dos retornos diários com o VaR foi preciso utilizar uma estrutura de Looping For, onde inicializamos uma variável (a) que receberá o valor do VaR e outra variável inicializada como zero e que ao longo do Loop receberá a contagem dos dias que a perda da carteira foi superior ao VaR (b).

In [12]:
#Transformei a coluna Retorno em lista a fim de facilitar sua manipulação no comando de Loop

a = var

Carteira = Posicao['Retorno'].tolist()
b=0
for i in Carteira:
    if(i>a):
        b+=1

#Dias em que a perda foi superior ao VaR estimado

b

14

Calculados os dias em que a perda superou o VaR, faremos agora uma proporção desta contagem com os dias totais observados na amostra e, com isso, encontramos uma porcentagem de aproximandamente 5% das observações onde a perda excedeu o VaR estimado.

In [11]:
Fora_var = (b/len(Posicao.index))
Fora_var


0.056

Logo, a partir dos cálculos realizados podemos observar que o modelo utilizado está dentro dos limites considerados, podendo ser utilizado para tomadas de decisões referentes a reservas de capitais, por exemplo, ou mesmo outras estratégias financeiras. 
No entanto, este modelo de certa forma ainda pode apresentar melhorias dado que podemos considerar uma superestimação deste, pois as observações que excedem o VaR ainda estão um pouco acima do nível de significância definido o que pode levar a uma alocação ineficiente de capital, fazendo com que reservas sejam realizadas com valores acima do necessário.

Como dito no início de estudo, um dos principais objetivos de sua realização foi a consolidação do aprendizado em programação na liguagem Python, bem como sua aplicação em um campo de conhecimento na área em que atuo profissionalmente. Ainda assim, seria preciso um maior número de informações para tomada de decisões mais assertivas ou até mesmo a utilização de outros modelos para cálculo do VaR, dado que o modelo de Simulação Histórica possui alguns aspectos que deixam seus resultados menos rigorosos em comparação com outros modelos.